In [1]:
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd
import numpy as np
from scipy.sparse.linalg import cg
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import numpy as np
import shap
import os
import sys
import glob
import time
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import sklearn.tree 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import shap
import xgboost
from sklearn.model_selection import train_test_split
import matplotlib.pylab as pl
import pandas as pd
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args



/home/codespace/.python/current/lib/python3.10/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/home/codespace/.python/current/lib/python3.10/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimizatio

In [2]:
# 导入数据

data = pd.read_csv('/Users/jiqiang/Nutstore Files/DULT/小论文/第一篇小论文/数据/jh_z/model training_pc_20230521/final_data_input_important_16869.csv')
# C:/Jiqiang/jh_z/jh_z/model training_pc/final_data_input_important.csv
# /Users/jiqiang/Nutstore Files/DULT/小论文/第一篇小论文/数据/jh_z/model training_pc_20230521/final_data_input_important.csv
df= pd.DataFrame(data)
counts = df['label'].value_counts()
num_zeros = counts[0]
num_ones = counts[1]
print("number of zero:",num_zeros)
print("number of one:",num_ones)


# 从总数据中抽取1000个1和1000个0作为训练和测试，剩下的作为预测集
# 随机抽取label为0和1的数据各1000个
df1 = pd.concat([df[df['label']==0].sample(n=1000),df[df['label']==1].sample(n=1000)]) # 训练集和测试集
print(df1)

# 删除已经抽取的数据，剩下的放入df2中
df2 = df.drop(df1.index) # 剩下的数据作为预测集
df2

# 固定训练集、测试集、预测集，将其分别输出到CSV文件中
df1.to_csv('train_test_data.csv', index=False)
df2.to_csv('predict_data.csv', index=False)

# 用df1中的数据划分训练集和测试集，输出到CSV文件中，随后分别为训练集和测试集输入特征进行标准化
from sklearn.model_selection import train_test_split

# 提取最后一列并命名为“y”
Y = df1.iloc[:, -1]
Y.name = 'y'


X_train_orgin, X_test_orgin, Y_train_orgin, Y_test_orgin = train_test_split(df1.drop('label', axis=1), df1['label'], test_size=0.2, random_state=42)

# 输出训练集和测试集
X_train_Y_train = pd.concat([X_train_orgin, Y_train_orgin],axis=1)
X_test_Y_test = pd.concat([X_test_orgin, Y_test_orgin],axis=1)

X_train_Y_train.to_csv('train_data.csv', index=False)
X_test_Y_test.to_csv('test_data.csv', index=False)

# 训练集和测试集的特征标准化
from sklearn.preprocessing import StandardScaler


# 提取除最后一列和第一列单元名以外的所有列并命名为“X”
X_train_orgin_dropFID = X_train_orgin.iloc[:, 1:]
X_test_orgin_dropFID = X_test_orgin.iloc[:, 1:]
X_pred = df2.iloc[:, 1:-1]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_orgin_dropFID)
X_test_scaled = scaler.transform(X_test_orgin_dropFID)
X_pred_scaled = scaler.transform(X_pred)

# 将标准化后的数据输出到新的数据框中
X_train = pd.DataFrame(X_train_scaled, columns=X_train_orgin_dropFID.columns, index=X_train_orgin_dropFID.index)
X_test = pd.DataFrame(X_test_scaled, columns=X_test_orgin_dropFID.columns,  index=X_test_orgin_dropFID.index)
X_pred = pd.DataFrame(X_pred_scaled, columns=X_pred.columns, index=X_pred.index)

Y_train = Y_train_orgin
Y_test = Y_test_orgin

# 模型训练
def print_test_accuracy(f):
    print("Accuracy = {0}%".format(100*np.sum(f(X_test) == Y_test)/len(Y_test)))
    time.sleep(0.5) # to let the print get out before any progress bars

def print_train_accuracy(f):
    print("Accuracy = {0}%".format(100*np.sum(f(X_train) == Y_train)/len(Y_train)))
    time.sleep(0.5) # to let the print get out before any progress bars
shap.initjs()

import xgboost as xgb
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc

# 将数据分为X_train, Y_train, X_test, Y_test

# 定义XGBoost模型
xgb_model = xgb.XGBClassifier()

# 定义参数空间
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01],
    'n_estimators': [50, 100]
}

# 定义5折交叉验证策略
cv = StratifiedKFold(n_splits=5)

# 使用GridSearchCV进行自动调参和交叉验证
grid_search_xgb = GridSearchCV(xgb_model,
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=cv,
                           n_jobs=-1)

grid_search_xgb.fit(X_train, Y_train)

# 输出每一折的训练集和测试集准确率
for i in range(5):
    print("Fold ", i+1)
    train_index, test_index = list(cv.split(X_train, Y_train))[i]
    print("Training Accuracy: ", accuracy_score(Y_train.iloc[train_index], grid_search_xgb.predict(X_train.iloc[train_index])))
    print("Testing Accuracy: ", accuracy_score(Y_train.iloc[test_index], grid_search_xgb.predict(X_train.iloc[test_index])))

# 输出模型在训练集和测试集上的精准率、召回率、F1、ROC曲线和AUC曲线平均值
print("Training Accuracy: ", accuracy_score(Y_train, grid_search_xgb.predict(X_train)).round(3))
print("Testing Accuracy: ", accuracy_score(Y_test, grid_search_xgb.predict(X_test)).round(3))
print("Training Precision: ", precision_score(Y_train, grid_search_xgb.predict(X_train)).round(3))
print("Testing Precision: ", precision_score(Y_test, grid_search_xgb.predict(X_test)).round(3))
print("Training Recall: ", recall_score(Y_train, grid_search_xgb.predict(X_train)).round(3))
print("Testing Recall: ", recall_score(Y_test, grid_search_xgb.predict(X_test)).round(3))
print("Training F1 Score: ", f1_score(Y_train, grid_search_xgb.predict(X_train)).round(3))
print("Testing F1Score: ", f1_score(Y_test, grid_search_xgb.predict(X_test)).round(3))

# 计算ROC曲线和AUC曲线
fpr_train, tpr_train, _ = roc_curve(Y_train, grid_search_xgb.predict_proba(X_train)[:, 1])
fpr_test, tpr_test, _ = roc_curve(Y_test, grid_search_xgb.predict_proba(X_test)[:, 1])
roc_auc_train = auc(fpr_train, tpr_train)
roc_auc_test = auc(fpr_test, tpr_test)

# 输出ROC曲线和AUC曲线平均值
print("Training ROC AUC: ", roc_auc_train.round(3))
print("Testing ROC AUC: ", roc_auc_test.round(3)) 

# nn
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc

# 定义多层感知机模型
mlp = MLPClassifier()

# 定义参数网格
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 100)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'adam'],
    'alpha': [0.0001, 0.001],
    'learning_rate': ['constant', 'adaptive']
}

# 定义5折交叉验证器
cv = StratifiedKFold(n_splits=5)

# 定义网格搜索器
grid_search_mlp = GridSearchCV(mlp, param_grid=param_grid,
                           cv=cv, scoring='roc_auc')

# 训练模型并进行自动调参
grid_search_mlp.fit(X_train, Y_train)

# 输出最佳参数组合和对应的AUC值
print('Best parameters:', grid_search_mlp.best_params_)
print('Best AUC:', grid_search_mlp.best_score_)

# 计算模型在训练集和测试集上的精准率、召回率、F1、ROC曲线和AUC曲线等指标，并打印结果
Y_train_pred = grid_search_mlp.predict(X_train)
Y_test_pred = grid_search_mlp.predict(X_test)

print('Training accuracy:', accuracy_score(Y_train, Y_train_pred))
print('Testing accuracy:', accuracy_score(Y_test, Y_test_pred))

print('Training precision:', precision_score(Y_train, Y_train_pred))
print('Testing precision:', precision_score(Y_test, Y_test_pred))

print('Training recall:', recall_score(Y_train, Y_train_pred))
print('Testing recall:', recall_score(Y_test, Y_test_pred))

print('Training F1 score:', f1_score(Y_train, Y_train_pred))
print('Testing F1 score:', f1_score(Y_test, Y_test_pred))

fpr_train, tpr_train, _ = roc_curve(Y_train, grid_search_mlp.predict_proba(X_train)[:, 1])
fpr_test, tpr_test, _ = roc_curve(Y_test, grid_search_mlp.predict_proba(X_test)[:, 1])

print('Training AUC:', auc(fpr_train, tpr_train))
print('Testing AUC:', auc(fpr_test, tpr_test))

# LR
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
import pandas as pd

# 定义逻辑回归模型和参数网格
model = LogisticRegression()
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}

# 使用GridSearchCV进行自动调参和5折交叉验证
grid_search_lr = GridSearchCV(model, param_grid=param_grid, cv=5)
grid_search_lr.fit(X_train, Y_train)

# 打印最佳参数和最佳得分
print('Best Parameters:', grid_search_lr.best_params_)
print('Best Score:', grid_search_lr.best_score_)

# 在训练集上评估模型
Y_train_pred = grid_search_lr.predict(X_train)
print('Training Accuracy:', accuracy_score(Y_train, Y_train_pred))
print('Training Precision:', precision_score(Y_train, Y_train_pred))
print('Training Recall:', recall_score(Y_train, Y_train_pred))
print('Training F1 Score:', f1_score(Y_train, Y_train_pred))

# 在测试集上评估模型
Y_test_pred = grid_search_lr.predict(X_test)
print('Testing Accuracy:', accuracy_score(Y_test, Y_test_pred))
print('Testing Precision:', precision_score(Y_test, Y_test_pred))
print('Testing Recall:', recall_score, Y_test_pred)
print('Testing F1 Score:', f1_score(Y_test, Y_test_pred))

# 绘制ROC曲线和计算AUC
fpr, tpr, thresholds = roc_curve(Y_test, grid_search_lr.predict_proba(X_test)[:, 1])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='ROC Curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

# 将LR, XGB, NN的模型性能进行对比

import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, auc

# 训练三个机器学习模型并预测测试集
model_lr = grid_search_lr
model_xgb = grid_search_xgb
model_mlp = grid_search_mlp

Y_test_pred1 = model_lr.predict(X_test)
Y_test_pred2 = model_xgb.predict(X_test)
Y_test_pred3 = model_mlp.predict(X_test)

# 计算评价指标
accuracy_train1 = accuracy_score(Y_train, model_lr.predict(X_train))
accuracy_train2 = accuracy_score(Y_train, model_xgb.predict(X_train))
accuracy_train3 = accuracy_score(Y_train, model_mlp.predict(X_train))

accuracy_test1 = accuracy_score(Y_test, model_lr.predict(X_test))
accuracy_test2 = accuracy_score(Y_test, model_xgb.predict(X_test))
accuracy_test3 = accuracy_score(Y_test, model_mlp.predict(X_test))

precision_train1 = precision_score(Y_train, model_lr.predict(X_train))
precision_train2 = precision_score(Y_train, model_xgb.predict(X_train))
precision_train3 = precision_score(Y_train, model_mlp.predict(X_train))

precision_test1 = precision_score(Y_test, Y_test_pred1)
precision_test2 = precision_score(Y_test, Y_test_pred2)
precision_test3 = precision_score(Y_test, Y_test_pred3)

recall_train1 = recall_score(Y_train, model_lr.predict(X_train))
recall_train2 = recall_score(Y_train, model_xgb.predict(X_train))
recall_train3 = recall_score(Y_train, model_mlp.predict(X_train))

recall_test1 = recall_score(Y_test, Y_test_pred1)
recall_test2 = recall_score(Y_test, Y_test_pred2)
recall_test3 = recall_score(Y_test, Y_test_pred3)

f1score_train1 = f1_score(Y_train, model_lr.predict(X_train))
f1score_train2 = f1_score(Y_train, model_xgb.predict(X_train))
f1score_train3 = f1_score(Y_train, model_mlp.predict(X_train))

f1score_test1 = f1_score(Y_test, Y_test_pred1)
f1score_test2 = f1_score(Y_test, Y_test_pred2)
f1score_test3 = f1_score(Y_test, Y_test_pred3)

# 将评价指标存储在字典中
results_train = {
    'Model': ['Model_lr', 'Model_xgb', 'Model_mlp'],
    'Accuracy (train)': [accuracy_train1, accuracy_train2, accuracy_train3],
    'Precision (train)': [precision_train1, precision_train2, precision_train3],
    'Recall (train)': [recall_train1, recall_train2, recall_train3],
    'F1 score (train)': [f1score_train1, f1score_train2, f1score_train3],
}
results_test = {
    'Model': ['Model_lr', 'Model_xgb', 'Model_mlp'],
    'Accuracy (test)': [accuracy_test1, accuracy_test2, accuracy_test3],
    'Precision (test)': [precision_test1, precision_test2, precision_test3],
    'Recall (test)': [recall_test1, recall_test2, recall_test3],
    'F1 score (test)': [f1score_test1, f1score_test2, f1score_test3],
}
# 将字典转换为 Pandas 数据框
df_results_train = pd.DataFrame(results_train)
df_results_test = pd.DataFrame(results_test)

# 打印结果
print(df_results_train)
print(df_results_test)



number of zero: 14402
number of one: 2466


,FID_,average_max_rainfall,average_2hr_rainfall,average_72hr_rainfall,distance to coast,EV,TWI,DTW,label
0,0,1.032337,0.173904,0.540760,1399.816133,3.449655,2032.268709,7.511351,1
1,1,1.036179,0.173910,0.540058,1452.270984,3.351578,2289.101516,7.397089,1
2,2,1.034249,0.173900,0.540413,1427.899396,3.355286,2170.271564,7.548316,1
3,3,1.040040,0.173918,0.539438,1487.814834,3.489952,2261.629379,8.816202,1
4,4,1.038104,0.173915,0.539721,1472.312893,3.421316,2262.551560,9.006661,1
...,...,...,...,...,...,...,...,...,...
16863,17497,0.973075,0.173802,0.558195,288.711012,2.471787,995.830005,5.545090,0
16864,17498,0.972212,0.173765,0.558101,301.895848,2.573513,1024.210310,7.424834,0
16865,17499,0.974931,0.173857,0.558490,242.731595,2.150453,890.199884,6.642009,0
16866,17500,0.976530,0.173897,0.558775,224.446030,2.170294,786.371904,6.314220,0


### 将三个模型预测准确的样本选出来分别放入一个样本集中，用于后续可解释性方法对比分析
1. 用训练好的模型预测剩余样本
2. 将所有样本（训练集、测试集、预测集）预测概率与是否预测正确输出，并于样本编号匹配
3. 挑选出预测正确的样本
4. 用不同可解释性方法对不同模型预测正确的样本进行可解释性分析

In [94]:
# 将训练、测试、预测结果与输入一起保存到output文件中
# 训练集
output_train_lr = pd.concat([X_train_orgin, Y_train_orgin, pd.Series(model_lr.predict_proba(X_train)[:,1], index=X_train_orgin.index,name='probability_1')], axis=1) # 保留预测为1的概率
outpt_train_xgb = pd.concat([X_train_orgin, Y_train_orgin, pd.Series(model_xgb.predict_proba(X_train)[:,1], index=X_train_orgin.index,name='probability_1')], axis=1) # 保留预测为1的概率
output_train_mlp = pd.concat([X_train_orgin, Y_train_orgin, pd.Series(model_mlp.predict_proba(X_train)[:,1], index=X_train_orgin.index, name='probability_1')], axis=1) # 保留预测为1的概率

# 测试集
output_test_lr = pd.concat([X_test_orgin, Y_test_orgin, pd.Series(model_lr.predict_proba(X_test)[:,1], index=X_test_orgin.index,name='probability_1')], axis=1) # 保留预测为1的概率
output_test_xgb = pd.concat([X_test_orgin, Y_test_orgin, pd.Series(model_xgb.predict_proba(X_test)[:,1], index=X_test_orgin.index,name='probability_1')], axis=1) # 保留预测为1的概率
output_test_mlp = pd.concat([X_test_orgin, Y_test_orgin, pd.Series(model_mlp.predict_proba(X_test)[:,1], index=X_test_orgin.index, name='probability_1')], axis=1) # 保留预测为1的概率

# 预测集
output_pred_lr = pd.concat([df2, pd.Series(model_lr.predict_proba(X_pred)[:,1], index=df2.index,name='probability_1')], axis=1) # 保留预测为1的概率
output_pred_xgb = pd.concat([df2, pd.Series(model_xgb.predict_proba(X_pred)[:,1], index=df2.index,name='probability_1')], axis=1) # 保留预测为1的概率
output_pred_mlp = pd.concat([df2, pd.Series(model_mlp.predict_proba(X_pred)[:,1], index=df2.index, name='probability_1')], axis=1) # 保留预测为1的概率


# predicted列为预测结果，0为未被淹，1为被淹
# 训练集
output_train_lr['predicted'] = (output_train_lr['probability_1'] >= 0.5).astype(int)
outpt_train_xgb['predicted'] = (outpt_train_xgb['probability_1'] >= 0.5).astype(int)
output_train_mlp['predicted'] = (output_train_mlp['probability_1'] >= 0.5).astype(int)

# 测试集
output_test_lr['predicted'] = (output_test_lr['probability_1'] >= 0.5).astype(int)
output_test_xgb['predicted'] = (output_test_xgb['probability_1'] >= 0.5).astype(int)
output_test_mlp['predicted'] = (output_test_mlp['probability_1'] >= 0.5).astype(int)

# 预测集
output_pred_lr['predicted'] = (output_pred_lr['probability_1'] >= 0.5).astype(int)
output_pred_xgb['predicted'] = (output_pred_xgb['probability_1'] >= 0.5).astype(int)
output_pred_mlp['predicted'] = (output_pred_mlp['probability_1'] >= 0.5).astype(int)


# 判断预测是否正确
# 训练集
output_train_lr['correct'] = (output_train_lr['predicted'] == output_train_lr['label']).astype(int)
outpt_train_xgb['correct'] = (outpt_train_xgb['predicted'] == outpt_train_xgb['label']).astype(int)
output_train_mlp['correct'] = (output_train_mlp['predicted'] == output_train_mlp['label']).astype(int)

# 测试集
output_test_lr['correct'] = (output_test_lr['predicted'] == output_test_lr['label']).astype(int)
output_test_xgb['correct'] = (output_test_xgb['predicted'] == output_test_xgb['label']).astype(int)
output_test_mlp['correct'] = (output_test_mlp['predicted'] == output_test_mlp['label']).astype(int)

# 预测集
output_pred_lr['correct'] = (output_pred_lr['predicted'] == output_pred_lr['label']).astype(int)
output_pred_xgb['correct'] = (output_pred_xgb['predicted'] == output_pred_xgb['label']).astype(int)
output_pred_mlp['correct'] = (output_pred_mlp['predicted'] == output_pred_mlp['label']).astype(int)

# 将预测正确的样本保存在output文件中
# 训练集
output_train_lr_correct = output_train_lr[output_train_lr['correct'] == 1]
outpt_train_xgb_correct = outpt_train_xgb[outpt_train_xgb['correct'] == 1]
output_train_mlp_correct = output_train_mlp[output_train_mlp['correct'] == 1]

# 测试集
output_test_lr_correct = output_test_lr[output_test_lr['correct'] == 1]
output_test_xgb_correct = output_test_xgb[output_test_xgb['correct'] == 1]
output_test_mlp_correct = output_test_mlp[output_test_mlp['correct'] == 1]

# 预测集
output_pred_lr_correct = output_pred_lr[output_pred_lr['correct'] == 1]
output_pred_xgb_correct = output_pred_xgb[output_pred_xgb['correct'] == 1]
output_pred_mlp_correct = output_pred_mlp[output_pred_mlp['correct'] == 1]

# 保存到CSV文件中
# 测试集
output_train_lr_correct.to_csv('/workspaces/Interpretable_ML_UFSM/Interpretable_ML_for_UFSM/output/output_train_lr_correct.csv', index=False)
outpt_train_xgb_correct.to_csv('/workspaces/Interpretable_ML_UFSM/Interpretable_ML_for_UFSM/output/output_train_xgb_correct.csv', index=False)
output_train_mlp_correct.to_csv('/workspaces/Interpretable_ML_UFSM/Interpretable_ML_for_UFSM/output/output_train_mlp_correct.csv', index=False)

# 测试集
output_test_lr_correct.to_csv('/workspaces/Interpretable_ML_UFSM/Interpretable_ML_for_UFSM/output/output_test_lr_correct.csv', index=False)
output_test_xgb_correct.to_csv('/workspaces/Interpretable_ML_UFSM/Interpretable_ML_for_UFSM/output/output_test_xgb_correct.csv', index=False)
output_test_mlp_correct.to_csv('/workspaces/Interpretable_ML_UFSM/Interpretable_ML_for_UFSM/output/output_test_mlp_correct.csv', index=False)

# 预测集
output_pred_lr_correct.to_csv('/workspaces/Interpretable_ML_UFSM/Interpretable_ML_for_UFSM/output/output_pred_lr_correct.csv', index=False)
output_pred_xgb_correct.to_csv('/workspaces/Interpretable_ML_UFSM/Interpretable_ML_for_UFSM/output/output_pred_xgb_correct.csv', index=False)
output_pred_mlp_correct.to_csv('/workspaces/Interpretable_ML_UFSM/Interpretable_ML_for_UFSM/output/output_pred_mlp_correct.csv', index=False)

# 将测试集、训练集、预测集结果预测正确的样本保存在一个文件中
output_pred_lr_correct_all = pd.concat([output_train_lr_correct, output_test_lr_correct, output_pred_lr_correct], axis=0)
output_pred_xgb_correct_all = pd.concat([outpt_train_xgb_correct, output_test_xgb_correct, output_pred_xgb_correct], axis=0)
output_pred_mlp_correct_all = pd.concat([output_train_mlp_correct, output_test_mlp_correct, output_pred_mlp_correct], axis=0)

# 保存到CSV文件中
output_pred_lr_correct_all.to_csv('/workspaces/Interpretable_ML_UFSM/Interpretable_ML_for_UFSM/output/all_output_pred_lr_correct.csv', index=False)
output_pred_xgb_correct_all.to_csv('/workspaces/Interpretable_ML_UFSM/Interpretable_ML_for_UFSM/output/all_output_pred_xgb_correct.csv', index=False)
output_pred_mlp_correct_all.to_csv('/workspaces/Interpretable_ML_UFSM/Interpretable_ML_for_UFSM/output/all_output_pred_mlp_correct.csv', index=False)

# 保存到CSV文件中
output_train_lr.to_csv('output_train_lr.csv', index=False)
outpt_train_xgb.to_csv('output_train_xgb.csv', index=False)
output_train_mlp.to_csv('output_train_mlp.csv', index=False)

